# The 10-Minute Modeling Tool

Let's build a modeling tool in 10 minutes!

We'll use [Neo4j](http://neo4j.com/) for data storage, [Python](https://www.python.org/) for data processing, [vis.js](http://visjs.org/) for data visualization.

## Neo4j

From [Wikipedia's Neo4j article](https://en.wikipedia.org/wiki/Neo4j) (with ephasis by me):

> Neo4j is an **open-source graph database** implemented in Java and **accessible from software written in other languages** using the **Cypher query language** through a transactional HTTP endpoint. The developers describe Neo4j as an ACID-compliant transactional database with native graph storage and processing. **Neo4j is the most popular graph database.**

A graph database is simply a set of **nodes** (things) and **relationships** (pairs of things). Both nodes and relationships can have labels or properties associated with them. Graph-traversal algorithms (both built-in and custom-built) read the properties of nodes and traverse the relationships between them to return nodes and relationships of interest.

## 10-second overview

In this session, we'll connect to the blank Neo4j database running on our local server.

Then, we'll populate it with nodes that will define:

* a "model element"
* a "model connection"
* our metamodel \*
* our project model
* our project data

and relationships that will define:

* how our metamodel \* conforms to the idea of "elements" and "connections"
* how our project model conforms to our metamodel \*
* how our project data conforms to our project model

Finally, we'll query our database and obtain useful views of our data, as well as suggestions for what additional data could be defined (what to ask our stakeholders).

\* We'll use a subset of UML in this session, but any metamodel is possible - SysML, UML, multi-layed ontologies, etc.

## Jack and Jill and Kidney Failure

Jack needs a kidney transplant or he will die. We will find him a donor with modeling!

## Connect to the Neo4j Database

First we'll connect to the database and delete any existing nodes and relationships.

We'll communicate with Neo4j through the `py2neo` Python library.

In [1]:
from py2neo import authenticate, Graph
from py2neo.error import Unauthorized

graph = Graph()

try:
    graph.delete_all()
except Unauthorized:
    # If you have a username/password, replace neo4j/password with your username/password
    authenticate('localhost:7474', 'neo4j', 'password')
    graph.delete_all()

## Element and Connection

Modeling is all about elements and connections. Similarly, graphs are all about nodes and relationships.

We'll draw a clear distinction between element/connection and node/relationship right now - **both elements and connections are nodes.**

##### Connections are Nodes?

Connections have a lot of information in them - a connection's source and target are specific elements, and its generalization and specializations are other specific connections.

Because a connection has relationships to those elements and connections, it cannot be a relationship itself. **Relationships do not have relationships.**

#### Let's define our first nodes!

In [2]:
from py2neo import Node
from scripts.vis import draw

# Neo4j nodes are defined by labels and properties
element = Node(
    'Element',      # label
    name='element'  # property
)
graph.create(element)

connection = Node('Connection', name='connection')
graph.create(connection)

# This function draws all of the graph's nodes and relationships in a bouncy vis.js visualization - drag the nodes around!
draw(graph)

Elements connect to other elements through connections, so elements are the sources and targets of connections.

We'll define relationships with a subject, verb, and object.

In the below cell, the relationships are **an element is the source of a connection** and **an element is the target of a connection**.

In [3]:
from py2neo import Relationship

# Neo4j relationships are mostly defined by a string label, as relationships do not inherit or relate to anything else.
IS_THE_SOURCE_OF = 'IS_THE_SOURCE_OF'
IS_THE_TARGET_OF = 'IS_THE_TARGET_OF'

graph.create(Relationship(element, IS_THE_SOURCE_OF, connection))
graph.create(Relationship(element, IS_THE_TARGET_OF, connection))

# Unfortunately, vis.js draws relationships on top of each other - "IS_THE_TARGET_OF" is underneath "IS_THE_SOURCE_OF".
draw(graph)

## Metamodel Elements

Now we need a metamodel - the logical framework that will eventually structure our model. For this project, we'll use a handful of elements and connections from UML, but the entirety of SysML can be defined in this same way.

We'll define Class, Abstract Class, and Interface

In [4]:
class_ = Node('Metamodel Element', name='class')
abstract_class = Node('Metamodel Element', name='abstract class')
interface = Node('Metamodel Element', name='interface')

graph.create(class_, abstract_class, interface)

draw(graph)

### Relating the metamodel

Class, Abstract Class, and Interface are all elements, so we relate the metamodel element nodes to the element node.

In [5]:
IS_A_TYPE_OF = 'IS_A_TYPE_OF'

graph.create(Relationship(class_,         IS_A_TYPE_OF, element))
graph.create(Relationship(abstract_class, IS_A_TYPE_OF, class_ ))
graph.create(Relationship(interface,      IS_A_TYPE_OF, element))

draw(graph)

## Metamodel Connections

The metamodel also contains connections between its elements, such as "a Class implements an Interface".

Let's create these connections and their relationships at the same time.

In [6]:
composes = Node('Metamodel Connection', name='composes')
graph.create(Relationship(composes, IS_A_TYPE_OF, connection))
graph.create(Relationship(class_, IS_THE_SOURCE_OF, composes))
graph.create(Relationship(class_, IS_THE_TARGET_OF, composes))

generalizes = Node('Metamodel Connection', name='generalizes')
graph.create(Relationship(generalizes, IS_A_TYPE_OF, connection))
graph.create(Relationship(class_, IS_THE_SOURCE_OF, generalizes))
graph.create(Relationship(class_, IS_THE_TARGET_OF, generalizes))

implements = Node('Metamodel Connection', name='implements')
graph.create(Relationship(implements, IS_A_TYPE_OF, connection))
graph.create(Relationship(class_,    IS_THE_SOURCE_OF, implements))
graph.create(Relationship(interface, IS_THE_TARGET_OF, implements))

draw(graph)

The relationships within the metamodel are highlighted in black.

Drag nodes around to get a better view of the relationships.

## Model Elements

Let's define our project model - in order to find Jack a kidney donor, we'll need to model the following:

* humans
* kidneys
* blood types
* friendship
* dogs
* octopuses

Obviously we need to model dogs and octopuses in order to be thorough about this kidney search.

In [7]:
animal = Node('Model Element', name='animal')
graph.create(Relationship(animal, IS_A_TYPE_OF, abstract_class))

vertebrate = Node('Model Element', name='vertebrate')
graph.create(Relationship(vertebrate, IS_A_TYPE_OF, interface))

human = Node('Model Element', name='human')
graph.create(Relationship(human, IS_A_TYPE_OF, animal))

dog = Node('Model Element', name='dog')
graph.create(Relationship(dog, IS_A_TYPE_OF, animal))

octopus = Node('Model Element', name='octopus')
graph.create(Relationship(octopus, IS_A_TYPE_OF, animal))

organ = Node('Model Element', name='organ')
graph.create(Relationship(organ, IS_A_TYPE_OF, abstract_class))

kidney = Node('Model Element', name='kidney')
graph.create(Relationship(kidney, IS_A_TYPE_OF, organ))

blood_type = Node('Model Element', name='blood type')
graph.create(Relationship(blood_type, IS_A_TYPE_OF, class_))

draw(graph, labels=['Model Element'])

The above visualization is only a subset of the entire graph.

From now on, after adding to the graph, we'll view it in multiple stages, zooming out from the new nodes to see the entire graph.

Zoom in/out (by scrolling), pan (by dragging the white background), and drag nodes around to get a better view of all these graphs.

In [8]:
draw(graph)

## Model Connections

To define how these model elements connect, we not only **sub-type existing** connection elements, we also **create new** connection elements.

"A human is a type of animal" is a connection that we define for model completeness, but "a dog has a blood type" exists to enable specific dogs and specific blood types to connect when we input our dog and blood type data.

In [9]:
animal__generalizes__human = Node('Model Connection', name='animal generalizes human')
graph.create(Relationship(animal__generalizes__human, IS_A_TYPE_OF, generalizes))
graph.create(Relationship(animal, IS_THE_SOURCE_OF, animal__generalizes__human))
graph.create(Relationship(human,  IS_THE_TARGET_OF, animal__generalizes__human))

animal__generalizes__dog = Node('Model Connection', name='animal generalizes dog')
graph.create(Relationship(animal__generalizes__dog, IS_A_TYPE_OF, generalizes))
graph.create(Relationship(animal, IS_THE_SOURCE_OF, animal__generalizes__dog))
graph.create(Relationship(dog,    IS_THE_TARGET_OF, animal__generalizes__dog))

animal__generalizes__octopus = Node('Model Connection', name='animal generalizes octopus')
graph.create(Relationship(animal__generalizes__octopus, IS_A_TYPE_OF, generalizes))
graph.create(Relationship(animal,  IS_THE_SOURCE_OF, animal__generalizes__octopus))
graph.create(Relationship(octopus, IS_THE_TARGET_OF, animal__generalizes__octopus))

human__implements__vertebrate = Node('Model Connection', name='human implements vertebrate')
graph.create(Relationship(human__implements__vertebrate, IS_A_TYPE_OF, implements))
graph.create(Relationship(human,      IS_THE_SOURCE_OF, human__implements__vertebrate))
graph.create(Relationship(vertebrate, IS_THE_TARGET_OF, human__implements__vertebrate))

dog__implements__vertebrate = Node('Model Connection', name='dog implements vertebrate')
graph.create(Relationship(dog__implements__vertebrate, IS_A_TYPE_OF, implements))
graph.create(Relationship(dog,        IS_THE_SOURCE_OF, dog__implements__vertebrate))
graph.create(Relationship(vertebrate, IS_THE_TARGET_OF, dog__implements__vertebrate))

organ__composes__animal = Node('Model Connection', name='organ is inside')
graph.create(Relationship(organ__composes__animal, IS_A_TYPE_OF, composes))
graph.create(Relationship(animal, IS_THE_SOURCE_OF, organ__composes__animal))
graph.create(Relationship(organ,  IS_THE_TARGET_OF, organ__composes__animal))

has_blood_type = Node('Model Connection', name='has blood type')
graph.create(Relationship(has_blood_type, IS_A_TYPE_OF, composes))
graph.create(Relationship(animal,     IS_THE_SOURCE_OF, has_blood_type))
graph.create(Relationship(blood_type, IS_THE_TARGET_OF, has_blood_type))

kidney_is_inside = Node('Model Connection', name='kidney is inside')
graph.create(Relationship(kidney_is_inside, IS_A_TYPE_OF, composes))
graph.create(Relationship(kidney,     IS_THE_SOURCE_OF, kidney_is_inside))
graph.create(Relationship(vertebrate, IS_THE_TARGET_OF, kidney_is_inside))

is_friends_with = Node('Model Connection', name='is friends with')
graph.create(Relationship(is_friends_with, IS_A_TYPE_OF, connection))
graph.create(Relationship(animal, IS_THE_SOURCE_OF, is_friends_with))
graph.create(Relationship(animal, IS_THE_SOURCE_OF, is_friends_with))

draw(graph, labels=['Model Connection'])

In [10]:
draw(graph, labels=['Model Connection', 'Model Element'])

In [11]:
draw(graph)

## Instance Element

Data time!

In [12]:
jack = Node('Instance Element', name='jack')
graph.create(Relationship(jack, IS_A_TYPE_OF, human))

jill = Node('Instance Element', name='jill')
graph.create(Relationship(jill, IS_A_TYPE_OF, human))

jasper = Node('Instance Element', name='jasper')
graph.create(Relationship(jasper, IS_A_TYPE_OF, dog))

jacoby = Node('Instance Element', name='jacoby')
graph.create(Relationship(jacoby, IS_A_TYPE_OF, octopus))

a_positive = Node('Instance Element', name='A+')
graph.create(Relationship(a_positive, IS_A_TYPE_OF, blood_type))

b_negative = Node('Instance Element', name='B-')
graph.create(Relationship(b_negative, IS_A_TYPE_OF, blood_type))

dea_one_point_one_negative = Node('Instance Element', name='dea 1.1-')
graph.create(Relationship(dea_one_point_one_negative, IS_A_TYPE_OF, blood_type))

invertebrate_blood = Node('Instance Element', name='invertebrate blood')
graph.create(Relationship(invertebrate_blood, IS_A_TYPE_OF, blood_type))

jill_kidney = Node('Instance Element', name="jill's kidney")
graph.create(Relationship(jill_kidney, IS_A_TYPE_OF, kidney))

jasper_kidney = Node('Instance Element', name="jasper's kidney")
graph.create(Relationship(jasper_kidney, IS_A_TYPE_OF, kidney))

draw(graph)

## Instance Connections

Wooooooooooooooooooo!

In [13]:
jack__has_blood_type__a_positive = Node('Instance Connection', name='jack has blood type A+')
graph.create(Relationship(jack__has_blood_type__a_positive, IS_A_TYPE_OF, has_blood_type))
graph.create(Relationship(jack,       IS_THE_SOURCE_OF, jack__has_blood_type__a_positive))
graph.create(Relationship(a_positive, IS_THE_TARGET_OF, jack__has_blood_type__a_positive))

jill__has_blood_type__b_negative = Node('Instance Connection', name='jill has blood type B-')
graph.create(Relationship(jill__has_blood_type__b_negative, IS_A_TYPE_OF, has_blood_type))
graph.create(Relationship(jill,       IS_THE_SOURCE_OF, jill__has_blood_type__b_negative))
graph.create(Relationship(b_negative, IS_THE_TARGET_OF, jill__has_blood_type__b_negative))

jasper__has_blood_type__dea_one_point_one_negative = Node('Instance Connection', name='jasper has blood type B-')
graph.create(Relationship(jasper__has_blood_type__dea_one_point_one_negative, IS_A_TYPE_OF, has_blood_type))
graph.create(Relationship(jasper,                     IS_THE_SOURCE_OF, jasper__has_blood_type__dea_one_point_one_negative))
graph.create(Relationship(dea_one_point_one_negative, IS_THE_TARGET_OF, jasper__has_blood_type__dea_one_point_one_negative))

jacoby__has_blood_type__invertebrate_blood = Node('Instance Connection', name='jacoby has blood type B-')
graph.create(Relationship(jacoby__has_blood_type__invertebrate_blood, IS_A_TYPE_OF, has_blood_type))
graph.create(Relationship(jacoby,             IS_THE_SOURCE_OF, jacoby__has_blood_type__invertebrate_blood))
graph.create(Relationship(invertebrate_blood, IS_THE_TARGET_OF, jacoby__has_blood_type__invertebrate_blood))

jack__is_friends_with__jill = Node('Instance Connection', name='jack is friends with jill')
graph.create(Relationship(jack__is_friends_with__jill, IS_A_TYPE_OF, is_friends_with))
graph.create(Relationship(jack, IS_THE_SOURCE_OF, jack__is_friends_with__jill))
graph.create(Relationship(jill, IS_THE_TARGET_OF, jack__is_friends_with__jill))

jack__is_friends_with__jasper = Node('Instance Connection', name='jack is friends with jasper')
graph.create(Relationship(jack__is_friends_with__jasper, IS_A_TYPE_OF, is_friends_with))
graph.create(Relationship(jack,   IS_THE_SOURCE_OF, jack__is_friends_with__jasper))
graph.create(Relationship(jasper, IS_THE_TARGET_OF, jack__is_friends_with__jasper))

jack__is_friends_with__jacoby = Node('Instance Connection', name='jack is friends with jacoby')
graph.create(Relationship(jack__is_friends_with__jacoby, IS_A_TYPE_OF, is_friends_with))
graph.create(Relationship(jack,   IS_THE_SOURCE_OF, jack__is_friends_with__jacoby))
graph.create(Relationship(jacoby, IS_THE_TARGET_OF, jack__is_friends_with__jacoby))

jill_kidney__kidney_is_inside__jill = Node('Instance Connection', name="jill's kidney is inside jill")
graph.create(Relationship(jill_kidney__kidney_is_inside__jill, IS_A_TYPE_OF, kidney_is_inside))
graph.create(Relationship(jill_kidney, IS_THE_SOURCE_OF, jill_kidney__kidney_is_inside__jill))
graph.create(Relationship(jill,        IS_THE_TARGET_OF, jill_kidney__kidney_is_inside__jill))

jasper_kidney__kidney_is_inside__jasper = Node('Instance Connection', name="jasper's kidney is inside jasper")
graph.create(Relationship(jasper_kidney__kidney_is_inside__jasper, IS_A_TYPE_OF, kidney_is_inside))
graph.create(Relationship(jasper_kidney, IS_THE_SOURCE_OF, jasper_kidney__kidney_is_inside__jasper))
graph.create(Relationship(jasper,        IS_THE_TARGET_OF, jasper_kidney__kidney_is_inside__jasper))

draw(graph, labels=['Instance Element'])

In [14]:
draw(graph, labels=['Instance Element', 'Model Element'])

In [15]:
draw(graph)